In [20]:
from surprise.dataset import Dataset

# загружаем встроенный набор данных
data = Dataset.load_builtin(name='ml-100k')

In [9]:
from surprise.model_selection import train_test_split

# разбиваем на обучающую и тестовую выборки
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [10]:
%%time
from surprise import KNNBasic

# обучаем модель
algo = KNNBasic(
    sim_options={
        'user_based': False
    }
)
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Wall time: 20.1 s


In [11]:
%%time
from surprise import SVD

#обучаем модель
algo = SVD(random_state=42)
algo.fit(trainset)

Wall time: 1min 6s


In [16]:
%%time
# получаем рекомендации валидационном наборе
predictions = algo.test(testset, verbose=0)

Wall time: 3.43 s


In [19]:
%%time
from surprise import accuracy

# оцениваем качество
accuracy.rmse(predictions)

RMSE: 0.8729
Wall time: 220 ms


0.8728931555711225

In [28]:
# возьмем определенного пользователя
# и посмотрим на его оценки
user_id = 42
user_ratings = trainset.ur[user_id]
print('список оценок:', user_ratings)
num_ratings = len(user_ratings)
print('всего оценок', num_ratings)
mean_rating = sum([rating for item_id, rating in user_ratings]) / num_ratings
print('средняя оценка:', mean_ratings)

список оценок: [(41, 3.0), (363, 4.0), (603, 3.0), (481, 5.0), (271, 5.0), (686, 5.0), (1068, 4.0), (1097, 4.0), (440, 5.0), (1603, 4.0), (151, 5.0), (1628, 5.0), (550, 5.0), (574, 5.0), (131, 5.0), (1570, 5.0), (112, 4.0), (265, 5.0), (1308, 3.0), (438, 5.0), (1562, 5.0), (520, 5.0), (828, 5.0), (210, 4.0), (63, 4.0), (934, 4.0), (394, 4.0), (832, 5.0), (325, 5.0), (2239, 4.0), (909, 4.0), (75, 4.0), (121, 5.0), (408, 5.0), (646, 5.0), (15, 5.0), (908, 5.0), (412, 5.0), (350, 5.0), (27, 5.0), (190, 5.0), (156, 5.0), (528, 5.0), (655, 4.0), (254, 5.0)]
всего оценок 45
средняя оценка: 4.5777777777777775


In [29]:
# посмотрим, как устроено предсказание
item_id = 41
# "внешнее API"  - сюда подаютсреальные ID пользователей и фильмов
print(algo.predict(
    uid=trainset.to_raw_uid(user_id),
    iid=trainset.to_raw_iid(item_id),
    r_ui=dict(trainset.ur[user_id])[item_id]
))
# внутреннее API - сюда подаются внутренние ID
print(algo.estimate(
    u=user_id,
    i=item_id
))
# формула предсказания
print(
    # скалярное произведелние скрытых факторов пользователя и фильма
    algo.pu[user_id].dot(algo.qi[item_id]) + 
    # смещение фильма
    algo.bi[item_id] +
    # смещение пользователя
    algo.bu[user_id] + 
    # общее смеение датасета
    trainset.global_mean
)

user: 288        item: 3398       r_ui = 3.00   est = 3.83   {'was_impossible': False}
3.8250928507915822
3.8250928507915822


In [30]:
import numpy as np

# скрытые факторы не нормированы
print(np.linalg.norm(algo.pu[user_id]))
# скрытые факторы не распределены ль 0 до 1
print(algo.pu.min())
print(algo.qi.min())
# или от -1 до 1
print(algo.pu.max())
print(algo.qi.max())
# среднее стандартное отклонение
print(algo.qi.mean())
print(algo.pu.mean())
print(algo.pu.std())
print(algo.qi.std())

1.1107987856767136
-1.06963803255275
-0.9803457306146611
1.0947878033759697
0.9315214141412997
-0.0006434778125401048
3.512544848782782e-05
0.13119236468642578
0.14139676994807773


In [32]:
# смещение пользователя не равно егшо средей оценке
print(mean_rating)
print(algo.bu[user_id])
print(algo.bu[user_id] + trainset.global_mean)

4.5777777777777775
0.5629317460462426
4.144421610037135
